In [ ]:
def count_lines(path):
    with open(path, "r") as f:
        for i, _ in enumerate(f, 1):
            pass
    return i

csv_path = "./resources/test.csv"
N = count_lines(csv_path)
print("Total rows:", N)


In [ ]:
import numpy as np
csv_path = "./resources/test.csv"
# N = 75922515-1
N = N-1
feature_dim = 32

features_mm = np.lib.format.open_memmap(
    "./resources/test_features.npy",
    dtype="float32",
    mode="w+",
    shape=(N, feature_dim)
)

# labels_mm = np.lib.format.open_memmap(
#     "./resources/labels.npy",
#     dtype="int64",
#     mode="w+",
#     shape=(N,)
# )


In [ ]:
import numpy as np

with open(csv_path, "r") as f:
    print(f.readline())
    for idx, line in enumerate(f):
        line = line.strip()
        if not line:
            continue

        values = line.split(",")
        if len(values) != 33:
            raise ValueError(f"Line {idx} malformed")

        # 转 float
        values = np.asarray(values, dtype=np.float32)

        # 丢弃 id，保留 32 维 feature
        features_mm[idx] = values[1:]

        # label 单独转 int
        # labels_mm[idx] = int(values[-1])

        if idx % 1_000_000 == 0 and idx > 0:
            print(f"Processed {idx} lines")

# 强制刷盘
features_mm.flush()
# labels_mm.flush()


In [ ]:
x = np.load("./resources/test_features.npy", mmap_mode="r")
y = np.load("./resources/labels.npy", mmap_mode="r", allow_pickle=True)

print(x.shape, x.dtype)
print(y.shape, y.dtype)

In [ ]:
import numpy as np

D = 32
count = 0
mean = np.zeros(D, dtype=np.float64)
M2 = np.zeros(D, dtype=np.float64)

with open(csv_path, "r") as f:
    f.readline()
    for line in f:
        values = np.fromstring(line, sep=",", dtype=np.float64)
        x = values[1:]   # (32,)

        count += 1
        delta = x - mean
        mean += delta / count
        delta2 = x - mean
        M2 += delta * delta2

var = M2 / (count - 1)
std = np.sqrt(var)

np.save("test_feature_mean.npy", mean.astype(np.float32))
np.save("test_feature_std.npy", std.astype(np.float32))


In [ ]:
import numpy as np
N = 766895-1
csv_path = "./resources/test.csv"
mean = np.load("test_feature_mean.npy")
std = np.load("test_feature_std.npy") + 1e-6

features_mm = np.lib.format.open_memmap(
    "./resources/test_features_norm.npy",
    mode="w+",
    dtype=np.float32,
    shape=(N, 32)
)

with open(csv_path, "r") as f:
    f.readline()
    for idx, line in enumerate(f):
        values = np.fromstring(line, sep=",", dtype=np.float32)
        x = values[1:]
        features_mm[idx] = (x - mean) / std

features_mm.flush()


In [ ]:
data_path = './resources/features_norm.npy'
label_path = './resources/labels.npy'

data = np.load(data_path, mmap_mode="r")[:10000]
np.save('./resources/val_features.npy', data)

labels = np.load(label_path, mmap_mode="r")[:10000]

np.save('./resources/val_labels.npy', labels)

In [ ]:
num_classes = 5
counts = np.bincount(labels.astype(np.int64), minlength=num_classes)
freq = counts / counts.sum()

print("Class counts:", counts)
print("Class freq:", freq)